# Removing Moiré pattern using notch filters

- Notch filters are bandreject filters with a narrow band of attenuated frequencies.
- In 2D, the stop band will be a small region around a *zero frequency* $(k_0,l_0)$.
- Notch filters have similar magnitude spectrum to highpass filters, except the center of their stop band is not at the origin $(0,0)$ but rather at $(k_0,l_0)$.
- Since we're only considering zero phase-shift filters that have *symmetric* spectrum, there will also be a stop band around $(-k_0,-l_0)$.
- Thus, notch filters can be expressed as products
  $$
  H_\textrm{NR}(k,l) = H_\textrm{HP}(k-k_0,l-l_0) \cdot H_\textrm{HP}(k+k_0,l+l_0)
  $$
  where $H_\textrm{HP}(k-k_0,l-l_0)$ is a highpass filter with its center shifted to $(k_0,l_0)$.
- If there are multiple stop bands, the joint notch filter is a product of partial notch filters, i.e.
  $$
  H_\textrm{NR}(k,l) = \prod_{b=1}^{B}{ H_\textrm{HP}(k-k_{0b},l-l_{0b}) \cdot H_\textrm{HP}(k+k_{0b},l+l_{0b}) }
  $$
  where $B$ is the number of stop bands.
- We'll use a notch filter to remove Moiré pattern from a printed image.

We'll proceed in four steps:
1. compute the DFT of the inputt image,
2. identify the Moiré harmonics as local maxima of the spectrum,
3. create a notch filter for each spectrum peak,
4. apply the notch filter to he image in the frequency domain and transform back to spatial domain.

<figure class="image">
  <img src="../figures/notch_filters-expected_outputs.png" alt="" style="width: 12.8in;"/>
  <figcaption>Figure 1: Expected outputs of the fake depth of field effect</figcaption>
</figure>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import skimage

In [ ]:
plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
gray = skimage.util.img_as_float(skimage.io.imread('../data/woman_moire.jpg', as_gray=True))
gray.shape, gray.dtype, gray.min(), gray.max()

# Task 1: Compute & show log of the centered DFT spectrum of the image

- Show the *logarithm* of the magnitude spectrum.
- The spectrum should be centered, i.e. the origin at the middle (`fftshift`).
- Plot the image and its spectrum.

# Task 2: Identify the frequencies to be zeroed out

- Find the spectrum peaks as local maxima of the magnitude.
- You can use the function [`skimage.feature.peak_local_max`](https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.peak_local_max).
- All the peaks except the one at the origin correspond to harmonics of the moiré pattern.
- Pick the indicies which are not the at the spectrum origin (all except one).
- Make sure that for each location $(k_0,l_0)$ you also pick its mirror peak $(-k_0,-l_0)$, i.e. the spectrum of the notch filter must be symmetric around the origin.
- Plot the results.

# Task 3: Create the notch filter

- At each location $(k_0, l_0)$, zero-out the spectrum using a narrow *Butterworth* high pass filter, i.e. *do not* zero-out using ideal high pass with sharp boundaries.
- You can use e.g. the `butterworth_lowpass2d` from the lecture. The function allows you to create a *shifted low pass* filter. Remember that HP = 1 - LP.
- The joint butterworth filter that zeroes-out all moiré frequencies simultaneously is the product of these single-frequency filters.
- Plot the transfer function of the joint notch filter, i.e. its magnitude spectrum.

# Task 4: Apply the filter to the image

- Convolve the Butterworth notch filter with the input image.
- E.g. you can use the `fftconvolve` function from the lecture or simply multiply the Butterworth filter (transfer function) with the input image's DFT spectrum and the `ifft2` back to the spatial domain.
- Be careful about whether the spectrums are centered (`fftshift`-ed) or not.
- Plot the original and filtered image next to each other.

# Task 5 (optional): Plot the notch filter in the spatial domain

- Transform the notch filter transfer function into the spatial domain and plot.
- You'll need to shift the origin of the kernel to the center of the image (e.g. using `np.roll`), see the lecture.
- Also, "zoom-in" on the center, i.e. plot only the middle part (again, see the lecture).